In [1]:
%cd lm_tab/scripts

/scratch/lgrinszt/lm_tab/scripts


In [2]:
import pandas as pd
cities = pd.read_csv("../data/cities.csv")

In [3]:
# rename columns
cities = cities.rename(columns={"Official est. GDP\nup to date\n(billion US$)": "gdp", "Metropolitan population": "pop", "Official est. GDP per capita": "gdp_capita",
                                "City proper/metropolitan area": "city", "Country/region": "country"})
cities = cities.drop(columns=["Rank"])

In [4]:
cities

,city,country,gdp,pop,gdp_capita
0,"Abbotsford, British Columbia",Canada,6.141 (2019),"202,497 (2019)","30,321.44"
1,Aberdeen,United Kingdom,23.0 (2020),"489,840 (2020)","46,957.94"
2,Abidjan,Ivory Coast,27 (2017),"5,950,000 (2022)","4,537.82"
3,Abu Dhabi,United Arab Emirates,119 (2015),"1,660,000 (2022)","71,686.75"
4,Adelaide,Australia,64.461 (2018–19),"1,380,000 (2022)","46,710.87"
...,...,...,...,...,...
714,Padua–Treviso–Venice metropolitan area,Italy,34.837 (2020),"2,748,420 (2020)","12,676.37"
715,Verona metropolitan area,Italy,34.152 (2020),"915,975 (2020)","37,282.68"
716,Heidelberg,Germany,34.038 (2020),"1,544,830 (2020)","22,034.79"
717,Murcia,Spain,33.923 (2020),"1,528,828 (2020)","22,186.93"


In [5]:
# Function to clean the data
import numpy as np
def clean_numeric_data(value):
    if isinstance(value, str):
        cleaned_str = value.split('(')[0].replace(',', '').replace(" ", "")
        try:
            return float(cleaned_str)
        except:
            print(f"Could not convert {value}, cleaned_str: {cleaned_str}")
            return np.nan
    return value

# Apply the function to the relevant columns
for col in ['gdp', 'pop', 'gdp_capita']:
    cities[col] = cities[col].apply(clean_numeric_data)

# drop rows with NaN values
cities = cities.dropna()

# Show the cleaned DataFrame
cities

Could not convert 300+,168 (2021), cleaned_str: 300+168
Could not convert 300+,851 (2021), cleaned_str: 300+851
Could not convert 300+,789 (2021), cleaned_str: 300+789


,city,country,gdp,pop,gdp_capita
0,"Abbotsford, British Columbia",Canada,6.141,202497.0,30321.44
1,Aberdeen,United Kingdom,23.000,489840.0,46957.94
2,Abidjan,Ivory Coast,27.000,5950000.0,4537.82
3,Abu Dhabi,United Arab Emirates,119.000,1660000.0,71686.75
4,Adelaide,Australia,64.461,1380000.0,46710.87
...,...,...,...,...,...
713,Ingolstadt,Germany,35.637,197560.0,180400.89
714,Padua–Treviso–Venice metropolitan area,Italy,34.837,2748420.0,12676.37
715,Verona metropolitan area,Italy,34.152,915975.0,37282.68
716,Heidelberg,Germany,34.038,1544830.0,22034.79


In [19]:
cities["name"] = cities["city"] + ", " + cities["country"]

/tmp/ipykernel_2474542/900666696.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
from src.data_loading import load_data
from skrub import MinHashEncoder
from sklearn.decomposition import PCA
from src.utils import FeaturesExtractor, FixedSizeSplit
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from tabpfn import TabPFNClassifier
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
import time
from sentence_transformers import SentenceTransformer

from sklearn.base import BaseEstimator, TransformerMixin


In [20]:
def generate_toy_problem(df, problem_type="matching"):
    cities = df.copy()
    if problem_type == "capita":
        cities["per_capita"] = np.random.uniform(0, 1, len(cities))
        cities["num"] = cities["pop"] * cities["per_capita"]
        X_rest = cities[["per_capita"]]
        X_rest_with_pop = cities[["per_capita", "pop"]]
        X_text = cities[["name"]]
        y = cities["num"] > np.median(cities["num"])
    elif problem_type == "matching":
        pop_shufled = cities["pop"].sample(frac=1).values
        choices = np.random.choice([0, 1], size=len(cities), p=[0.5, 0.5])
        # Based on the choices, create the 'new_pop' column
        cities['new_pop'] = np.where(choices == 0, cities['pop'], pop_shufled)
        # Store the choices as 'y'
        y = choices
        #X_rest = cities.drop(columns=["name", "city", "country", "gdp"])
        X_rest = cities[["new_pop"]]
        X_rest_with_pop = cities[["new_pop", "pop"]]
        X_text = cities[["name"]]
    return X_rest, X_rest_with_pop, X_text, y

In [14]:
from skrub import TableVectorizer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from  tabpfn import TabPFNClassifier
from sklearn.ensemble import GradientBoostingClassifier
from src.utils import run_on_encoded_data

models = {"TabPFNClassifier": TabPFNClassifier(device="cpu"), "GradientBoostingClassifier": GradientBoostingClassifier(), "LogisticRegression": LogisticRegression()}
dim_reductions = {#"subset_30": FeaturesExtractor(method="first", n_features=30),
                  "PCA_10": PCA(n_components=10)}
#                    "passthrough": "passthrough"}

cv = FixedSizeSplit(n_splits=5, n_train=380, n_test=300, random_state=42)


results = pd.DataFrame(columns=["dataset", "encoding", "dim_reduction", "model", "accuracy"])

dataset = "pop"
encoding = "lm__all-distilroberta-v1"
X_text = cities["name"]

for i in range(5):
    X_rest, X_rest_with_pop, X_text, y = generate_toy_problem(cities, problem_type=dataset)
    X_enc = encode(X_text, encoding)




    # # run with joblib
    results_data_enc = Parallel(n_jobs=1)(delayed(run_on_encoded_data)(X_enc, X_rest, y, dim_reduction_name, dim_reduction,model_name, model) for (dim_reduction_name, dim_reduction) in dim_reductions.items() for (model_name, model) in models.items())
    for dim_reduction_name, dim_reduction in dim_reductions.items():
        for model_name, model in models.items():
            scores = results_data_enc.pop(0)
            if scores is None:
                continue
            results = pd.concat([results, pd.DataFrame({"dataset": dataset, "encoding": encoding, "dim_reduction": dim_reduction_name, "model": model_name, "accuracy": scores, "features": "all"})])

    # same with X_rest = None
    results_data_enc = Parallel(n_jobs=1)(delayed(run_on_encoded_data)(X_enc, None, y, dim_reduction_name, dim_reduction,model_name, model) for (dim_reduction_name, dim_reduction) in dim_reductions.items() for (model_name, model) in models.items())
    for dim_reduction_name, dim_reduction in dim_reductions.items():
        for model_name, model in models.items():
            scores = results_data_enc.pop(0)
            if scores is None:
                continue
            results = pd.concat([results, pd.DataFrame({"dataset": dataset, "encoding": encoding, "dim_reduction": dim_reduction_name, "model": model_name, "accuracy": scores, "features": "text_only"})])

    results_data_enc = Parallel(n_jobs=1)(delayed(run_on_encoded_data)(None, X_rest, y, dim_reduction_name, dim_reduction,model_name, model) for (dim_reduction_name, dim_reduction) in dim_reductions.items() for (model_name, model) in models.items())
    for dim_reduction_name, dim_reduction in dim_reductions.items():
        for model_name, model in models.items():
            scores = results_data_enc.pop(0)
            if scores is None:
                continue
            results = pd.concat([results, pd.DataFrame({"dataset": dataset, "encoding": encoding, "dim_reduction": dim_reduction_name, "model": model_name, "accuracy": scores, "features": "rest_only"})])

    # add pop to X_rest
    results_data_enc = Parallel(n_jobs=1)(delayed(run_on_encoded_data)(X_enc, X_rest_with_pop, y, dim_reduction_name, dim_reduction,model_name, model) for (dim_reduction_name, dim_reduction) in dim_reductions.items() for (model_name, model) in models.items())
    for dim_reduction_name, dim_reduction in dim_reductions.items():
        for model_name, model in models.items():
            scores = results_data_enc.pop(0)
            if scores is None:
                continue
            results = pd.concat([results, pd.DataFrame({"dataset": dataset, "encoding": encoding, "dim_reduction": dim_reduction_name, "model": model_name, "accuracy": scores, "features": "all_plus_pop"})])
    
    # all rest + pop
    X_rest_with_pop = cities[["pop", "new_pop"]]
    X_enc_none = None
    results_data_enc = Parallel(n_jobs=1)(delayed(run_on_encoded_data)(None, X_rest_with_pop, y, dim_reduction_name, dim_reduction,model_name, model) for (dim_reduction_name, dim_reduction) in dim_reductions.items() for (model_name, model) in models.items())
    for dim_reduction_name, dim_reduction in dim_reductions.items():
        for model_name, model in models.items():
            scores = results_data_enc.pop(0)
            if scores is None:
                continue
            results = pd.concat([results, pd.DataFrame({"dataset": dataset, "encoding": encoding, "dim_reduction": dim_reduction_name, "model": model_name, "accuracy": scores, "features": "rest_plus_pop"})])



Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_2474542/3570328819.py:80: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2474542/3570328819.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Encoded data shape: (682, 768)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
(682, 768) (682, 1) (682, 769)
(682, 11)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_2474542/3570328819.py:80: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2474542/3570328819.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Encoded data shape: (682, 768)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
(682, 768) (682, 1) (682, 769)
(682, 11)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


...

Encoded data shape: (682, 768)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
(682, 768) (682, 1) (682, 769)
(682, 11)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_2474542/3570328819.py:80: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2474542/3570328819.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Encoded data shape: (682, 768)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
(682, 768) (682, 1) (682, 769)
(682, 11)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_2474542/3570328819.py:80: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_2474542/3570328819.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Encoded data shape: (682, 768)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
(682, 768) (682, 1) (682, 769)
(682, 11)
(682, 768) (682, 1) (682, 769)
(682, 11)
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


In [18]:
0.75*0.75 + 0.25*0.25

0.625

In [15]:
len(results)

375

In [16]:
# plot results
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


#melted_results = results.groupby(['dataset', 'model', 'dim_reduction', 'encoding', "features"]).mean().reset_index()
melted_results = results
#melted_results = results.explode('accuracy')
#melted_results['accuracy'] = melted_results['accuracy'].astype(float)
#melted_results = melted_results[melted_results['encoding'] == 'lm__all-MiniLM-L12-v2']
print(len(melted_results))

# Creating the swarmplot
# plt.figure(figsize=(15, 20))
# sns.swarmplot(data=melted_results, x='accuracy', y='dataset', hue='model', dodge=True)
# plt.title('Swarm Plot of Model Accuracies Across Datasets')
# plt.xlabel('Accuracy')
# plt.ylabel('Dataset')
# plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.show()
# Create the plot

# Create the plot
fig = px.strip(
    data_frame=melted_results,
    x="accuracy",
    y="features",
    color="model",
    #color="dim_reduction",
    title="Swarm Plot of Model Accuracies Across Datasets",
    labels={"accuracy": "Accuracy", "dataset": "Dataset", "model": "Model"},
    height=600,
    width=900,
)
fig.show()

375


In [13]:
results

,dataset,encoding,dim_reduction,model,accuracy,features
0,pop,lm__all-distilroberta-v1,PCA_10,TabPFNClassifier,0.586667,all
1,pop,lm__all-distilroberta-v1,PCA_10,TabPFNClassifier,0.626667,all
2,pop,lm__all-distilroberta-v1,PCA_10,TabPFNClassifier,0.646667,all
3,pop,lm__all-distilroberta-v1,PCA_10,TabPFNClassifier,0.553333,all
4,pop,lm__all-distilroberta-v1,PCA_10,TabPFNClassifier,0.640000,all
...,...,...,...,...,...,...
0,pop,lm__all-distilroberta-v1,PCA_10,LogisticRegression,0.466667,all_plus_pop
1,pop,lm__all-distilroberta-v1,PCA_10,LogisticRegression,0.490000,all_plus_pop
2,pop,lm__all-distilroberta-v1,PCA_10,LogisticRegression,0.473333,all_plus_pop
3,pop,lm__all-distilroberta-v1,PCA_10,LogisticRegression,0.483333,all_plus_pop
